# Leonardo's Variable Pitch Mechanism: Swashplate Dynamics

> *"Mechanical perfection is achieved through patience and precision. Every component must move with the grace of a bird's wing in flight."*  
> — Leonardo da Vinci

## Introduction
Leonardo's designs often featured complex linkages. This notebook explores a **Variable Pitch Swashplate**, a mechanism that changes the angle of blades (pitch) while they are rotating. This is essential for controlling lift and thrust in modern helicopters and propellers.

We will model:
1.  **Mechanical Advantage**: How levers amplify force.
2.  **Control Response**: How quickly the pitch changes.
3.  **Forces**: The loads on the swashplate.

---

In [ ]:
# Install the davinci-codex library
!pip install -q git+https://github.com/Shannon-Labs/davinci-codex.git

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

# Set "Brutalist Academic" plotting style
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman', 'DejaVu Serif'],
    'axes.grid': True,
    'grid.alpha': 0.3,
    'axes.facecolor': 'white',
    'figure.facecolor': 'white',
    'text.color': 'black',
    'axes.labelcolor': 'black',
    'xtick.color': 'black',
    'ytick.color': 'black'
})


## The Physics Model

### 1. Mechanical Advantage
The swashplate acts as a lever. The mechanical advantage (MA) is:
$$ MA = \frac{L_{input}}{L_{output}} \cos(\theta) $$

### 2. Dynamics
We model the pitch change as a second-order system (mass-spring-damper):
$$ I \ddot{\theta} + c \dot{\theta} + k \theta = \tau_{actuator} - \tau_{load} $$

Where:
- $I$: Moment of inertia.
- $\tau_{load}$: Aerodynamic and centrifugal loads.


In [ ]:
def simulate_pitch_control(target_pitch_deg, actuator_force_N):
    dt = 0.01
    time = np.arange(0, 2.0, dt)
    
    # System Parameters
    I = 0.05  # Moment of inertia (kg m^2)
    damping = 0.5
    
    # State
    pitch = np.zeros_like(time)
    pitch[0] = 15.0 # Start at 15 degrees
    velocity = 0.0
    
    # Control Loop (PD Controller)
    kp = 20.0
    kd = 2.0
    
    force_history = []
    
    for i in range(1, len(time)):
        current_pitch = pitch[i-1]
        error = target_pitch_deg - current_pitch
        
        # Desired force from controller
        desired_force = kp * error - kd * velocity
        
        # Limit force by actuator capability
        actual_force = np.clip(desired_force, -actuator_force_N, actuator_force_N)
        force_history.append(actual_force)
        
        # Dynamics
        torque = actual_force * 0.1  # 10cm lever arm
        accel = (torque - damping * velocity) / I
        
        velocity += accel * dt
        pitch[i] = pitch[i-1] + velocity * dt
        
    return time, pitch, force_history

def plot_mechanism(target_pitch, max_force):
    t, pitch, force = simulate_pitch_control(target_pitch, max_force)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot 1: Pitch Response
    ax1.plot(t, pitch, 'k-', linewidth=2, label='Blade Pitch')
    ax1.axhline(target_pitch, color='r', linestyle='--', label='Target')
    ax1.set_title('Pitch Response')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Pitch Angle (deg)')
    ax1.legend()
    
    # Plot 2: Actuator Force
    ax2.plot(t[1:], force, 'b-', label='Actuator Force')
    ax2.axhline(max_force, color='r', linestyle=':', label='Max Capacity')
    ax2.axhline(-max_force, color='r', linestyle=':')
    ax2.set_title('Control Effort')
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Force (N)')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    settling_time = t[np.where(np.abs(pitch - target_pitch) < 0.5)[0][0]] if np.any(np.abs(pitch - target_pitch) < 0.5) else 2.0
    print(f"Settling Time: {settling_time:.2f} s")

interact(plot_mechanism, 
         target_pitch=widgets.FloatSlider(min=15, max=45, step=1, value=30, description='Target Pitch (deg)'),
         max_force=widgets.IntSlider(min=10, max=100, step=5, value=50, description='Max Force (N)'));

## Conclusion

The simulation shows that a simple mechanical linkage can effectively control blade pitch. However, the **force required** increases with the speed of actuation. Leonardo's challenge would have been manufacturing bearings precise enough to reduce friction, allowing a human pilot to operate this mechanism while also powering the flight.